In [1]:
import numpy as np
from stochastic.processes.continuous import FractionalBrownianMotion
from scipy.special import gamma
import multiprocessing as mp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time


def GenerateNoise(H, t_fin, t_init, N, alpha, A, rng):
    '''the parameter rng produces unique series of random numbers at each simulation trajectory'''
    fbm = FractionalBrownianMotion(hurst=1-H, t=(t_fin - t_init), rng=rng)
    noise = np.sqrt(2 / gamma(1 + alpha)) / A * fbm.sample(N)
#     print(f"{noise[1]}\n")
    return noise


def force_term(j, x):
    return x[j-1]


def approxfun(n, alpha, j, x):
    return float(force_term(j,x)*(((n-j+1)**alpha) - ((n-j) ** alpha)))


def simulate_process(args):
    H, t_fin, t_init, N, alpha, A, dt, r, xr, m, w, seed, i = args

    # random number generator with a unique seed
    rng = np.random.default_rng(seed)
    coefficient = (m*(w**2)*(dt**alpha))/(gamma(1+alpha))
    noise = GenerateNoise(H, t_fin, t_init, N, alpha, A, rng)

    # defining the position array
    x = np.zeros(N)
    xm = 0
    tr = 0
    reset_step = 1
    count = 0
    nn = []

    for n in range(1, N):
        r1 = rng.uniform(0, 1)
        if r1 < r * dt:
            x[n] = xr
            xm = xr
            reset_time = n*dt
            reset_step = n
            noise = GenerateNoise(H, t_fin, t_init+reset_time, N-reset_step, alpha, A, rng)
            count += 1
        else:
            x[n] = xm + x[0] - coefficient*sum(approxfun(n, alpha, j, x) for j in range(reset_step, n+1)) + noise[n-reset_step]


    return x

if __name__ == "__main__":
    r = 1.0      # reset rate 
    xr = 1.0   # reset position 
    m = 1.0    # mass 
    w = 1.0    # barrier frequency 
    zeta = 1.0      # friction coefficient 
    H = 0.65        # Hurst index 
    N = 50000       # simulation steps 
    nreps = 10000   # number of simulations/trajectories 
    t_init = 0.0    # initial time 
    t_fin = 100.0   # final time 
    dt = (t_fin - t_init) / N   # time interval
    print(f'Time interval = {dt} sec.')
    alpha = 2-2*H
    A = 2*zeta*H*(2*H-1)*gamma(1-alpha)
    t1 = time.time()
    traj = np.zeros((nreps,N)) 
    
    with mp.Pool(processes=56) as pool:
        # Generate unique seeds for each process
        base_seed = int(time.time())
        seeds = [base_seed + i for i in range(nreps)]
        args = [(H, t_fin, t_init, N, alpha, A, dt, r, xr, m, w, seeds[i], i) for i in range(nreps)]
        trajs = pool.map(simulate_process, args)

Time interval = 0.002 sec.


In [3]:
df = pd.DataFrame(trajs)
df

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
0,0.0,0.0,-0.139286,-0.072801,-0.124669,0.215894,0.201330,0.210569,0.435466,0.413736,...,0.046554,-0.082327,-0.138270,-0.151100,-0.130875,-0.366150,-0.080722,0.100019,-0.008938,-0.226382
1,0.0,0.0,-0.045171,0.028616,-0.191524,-0.022972,-0.113727,-0.066117,-0.029375,0.052279,...,0.506266,0.476381,0.728999,0.749348,0.764387,0.636886,0.783110,0.897712,0.887548,0.746429
2,0.0,0.0,-0.075811,-0.241584,-0.334215,-0.248153,-0.101150,-0.030700,0.179102,0.242299,...,-0.018539,0.059868,0.032777,0.161451,-0.099088,0.416926,0.428615,0.230449,0.047595,-0.129236
3,0.0,0.0,-0.103223,-0.037737,-0.071936,-0.193064,-0.126528,0.085365,0.309366,0.033159,...,1.444351,1.619549,1.241097,1.379480,1.483721,1.231669,1.166761,1.173201,1.289777,1.425260
4,0.0,0.0,-0.220491,-0.178926,0.072307,-0.028866,-0.031482,0.148361,-0.118782,-0.233068,...,0.441040,0.480142,0.673077,0.605209,0.609214,0.799992,0.771773,0.811368,0.663578,0.748119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.206908,-0.024882,0.256191,0.207092,0.187652,0.360772,0.218048,0.144827,...,0.402173,0.392618,0.217295,0.183882,0.229964,0.422944,0.305109,0.256980,0.580358,0.610604
9996,0.0,0.0,0.115929,0.255728,0.290217,0.386040,0.433693,0.417112,0.143863,0.119677,...,-0.710908,-0.493966,-0.633289,-0.541069,-0.487325,-0.561988,-0.637279,-0.673420,-0.902789,-1.001346
9997,0.0,0.0,0.084814,0.110498,0.087825,-0.183131,-0.121397,-0.082773,-0.223330,-0.274260,...,-1.385396,-1.497489,-1.490304,-1.395739,-1.524237,-1.651167,-1.431746,-1.360637,-1.503933,-1.475593
9998,0.0,0.0,0.100235,-0.110509,-0.219912,-0.357203,-0.586521,-0.398974,-0.376874,-0.403199,...,0.332885,0.110460,0.289465,0.069848,0.042002,0.347110,0.105119,-0.051205,0.043982,0.116594
